# GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-1787463b-ae37-0b5a-0500-73088ca9171f)


# Drive Mount

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# MMCV install

In [3]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.1/index.html
!git clone https://github.com/open-mmlab/mmpose.git

Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.1/index.html
     |████████████████████████████████| 317kB 9.0MB/s 
     |████████████████████████████████| 194kB 16.3MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.8-cp37-cp37m-linux_x86_64.whl size=29318585 sha256=e092c65aecd4a086b676bbe64b01235c5dbcaa6ac9a93e805b0f241f1399ce53
  Stored in directory: /root/.cache/pip/wheels/5a/0d/c3/627bb7a31df052473f9edd2812702d4a0b2e302cb6932c9a91
Successfully built mmcv-full
Cloning into 'mmpose'...
remote: Enumerating objects: 11101, done.
remote: Counting objects: 100% (2990/2990), done.
remote: Compressing objects: 100% (1133/1133), done.
remote: Total 11101 (delta 2129), reused 2351 (delta 1817), pack-reused 8111
Receiving objects: 100% (11101/11101), 20.74 MiB | 29.42 MiB/s, done.
Resolving deltas: 100% (7376/7376), done.


# MMPOSE install

In [4]:
%cd mmpose/
!pip install -r requirements.txt
!python setup.py develop

/content/mmpose
Ignoring dataclasses: markers 'python_version == "3.6"' don't match your environment
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 317kB 23.3MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
  Using cached https://files.pythonhosted.org/packages/f4/f5/6605d73bf3f4c198915872111b10c4b3c2dccd8485f47b7290ceef037190/pytest_runner-5.3.1-py3-none-any.whl
     |████████████████████████████████| 122kB 48.5MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 12.2MB 258kB/s 
     |████████████████████████████████| 4.5MB 40.0MB/s 
  Cloning https://github.com/svenkreiss/poseval.git to /tmp/pip-install-ndz6n7er/poseval
  Running command git clone -q https://github.com/svenkreiss/poseval.git /tmp/pip-install-ndz6n7er/poseval
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 

# numpy & opencv update

In [5]:
!pip uninstall numpy -y
!pip uninstall opencv-python -y
!pip install numpy
!pip install --upgrade pip
!pip install opencv-python==4.0.0.21

Uninstalling numpy-1.19.5:
  Successfully uninstalled numpy-1.19.5
Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 15.7MB 194kB/s 
ERROR: imgaug 0.2.9 requires opencv-python, which is not installed.
ERROR: dopamine-rl 1.0.5 requires opencv-python>=3.4.1.15, which is not installed.
ERROR: mmpose 0.16.0 requires opencv-python, which is not installed.
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 1.6MB 7.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 25.4 MB 83 kB/s 


# ^^ **restart the runtime** ^^

# Imports

In [3]:
import numpy as np
import shutil, os
import cv2

from tqdm import tqdm

from argparse import ArgumentParser,Namespace

#from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         #vis_pose_result)

from matplotlib import image
from matplotlib import pyplot as plt
%matplotlib inline 
plt.rcParams['figure.figsize'] = [10,10]

# Important Directories

In [19]:
zip_file = '/content/drive/.shortcut-targets-by-id/10MbzXPgqUxsh3avgtcKxDVUMb4tAKl-k/Keypoint to GCN/Dataset/AUTSL_zipped/train/train_set_vfbha39.zip'
unzip_directory = '/content/Dataset/test_vid/'
#output_directory = '/content/drive/.shortcut-targets-by-id/10MbzXPgqUxsh3avgtcKxDVUMb4tAKl-k/Keypoint to GCN/Dataset/test_npy_files_Darkpose/'
output_directory = '/content/testing/'

## extra depth for unzipped file
all_files_dirctory = os.path.join(unzip_directory,'train_set_vfbha39','train') 
all_files_dirctory

'/content/Dataset/test_vid/train_set_vfbha39/train'

# Unzipping

In [7]:
shutil.unpack_archive(zip_file,unzip_directory)

OSError: ignored

# Counting Files

In [20]:
ls = os.listdir(all_files_dirctory)
ls.sort()
len(ls)

46021

# Counting Signers

In [21]:
dc = {}
for l in ls:
    t = l.split('_')
    if t[2] != 'depth.mp4':
        if t[0] not in dc.keys():
            dc[t[0]] = 0
        dc[t[0]] += 1

for k in dc.keys():
    print(f"{k} \t: {dc[k]}")
print(f'Total {len(dc)} signer')

signer0 	: 1350
signer10 	: 669
signer12 	: 2026
signer13 	: 2022
signer15 	: 448
signer17 	: 441
signer19 	: 641
signer20 	: 664
signer21 	: 632
signer22 	: 627
signer23 	: 678
signer24 	: 669
signer26 	: 645
signer28 	: 640
signer29 	: 643
signer2 	: 1356
signer31 	: 662
signer32 	: 629
signer33 	: 660
signer36 	: 654
signer37 	: 650
signer38 	: 641
signer3 	: 672
signer40 	: 662
signer41 	: 673
signer42 	: 672
signer4 	: 672
signer5 	: 678
signer7 	: 935
Total 29 signer


# Removing depth file and other signers

In [43]:
ls = os.listdir(all_files_dirctory)
ls.sort()
#################################################################
signer = 6          #keeping this signer
#################################################################
for l in ls:
    parts = l.split('_')
    if 'depth.mp4' in parts:
        pat = os.path.join(all_files_dirctory,l)
        #print(pat)
        os.remove(pat)
    
    elif f'signer{signer}' not in parts:
        pat = os.path.join(all_files_dirctory,l)
        #print(pat)
        os.remove(pat)


ls = os.listdir(all_files_dirctory)
ls.sort()
len(ls)

678

In [45]:
%cd /content
!pwd
ls[0]

/content
/content


'signer6_sample100_color.mp4'

# Main Function

In [46]:
def my_main(video_path,output_path):
    #args = parser.parse_args()
    args = Namespace(device='cuda:0',
                     kpt_thr=0.3, 
                     pose_checkpoint='https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_384x288_dark-f5726563_20200918.pth', 
                     pose_config='mmpose/configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/hrnet_w48_coco_wholebody_384x288_dark_plus.py', 
                     video_path=video_path)

    #assert args.show or (args.out_video_root != '')
    # build the pose model from a config file and a checkpoint file
    pose_model = init_pose_model(
        args.pose_config, args.pose_checkpoint, device=args.device.lower())

    dataset = pose_model.cfg.data['test']['type']

    cap = cv2.VideoCapture(args.video_path)
    assert cap.isOpened(), f'Faild to load video file {args.video_path}'

    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    

    # optional
    return_heatmap = False

    # e.g. use ('backbone', ) to return backbone feature
    output_layer_names = None

    
    output_list = []
    while (cap.isOpened()):
        flag, img = cap.read()
        if not flag:
            break

        img = cv2.flip(img, flipCode=1)

        # keep the person class bounding boxes.
        person_results = [{'bbox': np.array([0, 0, size[0], size[1]])}]

        # test a single image, with a list of bboxes.
        pose_results, returned_outputs = inference_top_down_pose_model(
            pose_model,
            img,
            person_results,
            format='xyxy',
            dataset=dataset,
            return_heatmap=return_heatmap,
            outputs=output_layer_names)

        output_list.append(pose_results[0]['keypoints'])

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        

    cap.release()
    cv2.destroyAllWindows()
    output_list = np.array(output_list)
    np.save(output_path, output_list)
    


for l in tqdm(ls):
    vp = os.path.join(all_files_dirctory,l)
    op = os.path.join(output_directory,f'{l[:-4]}.npy')
    print(l[:-4],end = ' ->')
    my_main(vp,op)






  0%|          | 0/678 [00:00<?, ?it/s]

signer6_sample100_color ->Use load_from_http loader







  0%|          | 1/678 [00:12<2:25:07, 12.86s/it]

signer6_sample101_color ->Use load_from_http loader







  0%|          | 2/678 [00:21<2:09:02, 11.45s/it]

signer6_sample102_color ->Use load_from_http loader


KeyboardInterrupt: ignored